In [1]:
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from PIL import Image

ckpt_dir = "PaddleOCR-VL-SFT-test01"  # 저장된 체크포인트 디렉토리

# 모델/토크나이저/이미지 프로세서 로드
model = PaddleOCRVLForConditionalGeneration.from_pretrained(ckpt_dir)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(ckpt_dir, padding_side="right", model_max_length=4096)
image_processor = SiglipImageProcessor.from_pretrained(ckpt_dir)

# 입력 준비
img = Image.open("/path/to/your/image.jpg").convert("RGB")
prompt = "다음 이미지의 텍스트를 읽어줘."

# 간단한 전처리/토크나이즈 (프로젝트 파이프라인과 동일하게 맞추면 가장 좋습니다)
inputs = {
    "context": [
        {"role": "user", "utterance": [
            {"type": "text", "text": prompt},
            {"type": "image_url", "image_url": {"url": img, "image_width": img.width, "image_height": img.height}},
        ]}
    ]
}

# 프로젝트의 End2EndProcessor를 사용할 수도 있습니다.
from data_processor.steps.end2end_processing import End2EndProcessor
processor = End2EndProcessor(
    args=type("A", (), {"batch_size":1, "load_args_from_api":False, "max_seq_length":4096, "im_prefix_length":64,
                        "variable_resolution":1, "spatial_conv_size":1, "adaptive_max_imgtoken_option":None,
                        "adaptive_max_imgtoken_rate":None, "video_min_pixels":None, "video_max_pixels":None,
                        "drop_untrainble_sample":False, "chat_template":"ernie_vl"})(),
    tokenizer=tokenizer,
    image_preprocess=image_processor
)
features = processor.process(inputs)[0]

# 모델 포워드 (loss 없이 디코딩/생성)
model.eval()
with paddle.no_grad():
    output = model.generate(
        input_ids=paddle.to_tensor([features["ds16"]], dtype="int64"),
        token_type_ids=paddle.to_tensor([features["ds16_tokenwise_type_id"]], dtype="int64"),
        image_type_ids=paddle.to_tensor([features["ds16_imagewise_type_id"]], dtype="int64"),
        images=features["meta"][0]["images"],  # 프로젝트 내부 포맷에 맞춰 전달
        max_new_tokens=128,
    )
text = tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
print(text)

/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
W1110 14:35:41.506570 1225972 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 12.6, Runtime API Version: 11.8
[2025-11-10 14:35:43,031] [    INFO] - Using download source: huggingface
[2025-11-10 14:35:43,032] [    INFO] - Loading configuration file PaddleOCR-VL-SFT-test01/config.json
/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddleformers/transformers/model_utils.py:394: SyntaxWarning: invalid escape sequence '\.'
  if re.search(f"\.{trans_key}\.weight$", key) or re.fullmatch(f"^{trans_key}\.weight$", key):
/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddlefor

OSError: Error no file named model_state.pdparams, found in directory PaddleOCR-VL-SFT-test01.

In [4]:
model_path

'meister1378/vl-lora'

In [2]:
from transformers import AutoModelForCausalLM, AutoProcessor
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

preprocessor_config.json:   0%|          | 0.00/509 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

OSError: meister1378/vl-lora does not appear to have a file named processing_ppocrvl.py. Checkout 'https://huggingface.co/meister1378/vl-lora/tree/main' for available files.

In [5]:
import paddle
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel

base_model = "PaddlePaddle/PaddleOCR-VL"  # 학습 시 사용한 베이스와 동일해야 함
lora_dir   = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test01"  # peft_model*.safetensors 위치

# 1) 베이스 모델을 HF→Paddle 변환 로딩
model = PaddleOCRVLForConditionalGeneration.from_pretrained(
    base_model,
    convert_from_hf=True,      # 핵심: HF 포맷을 Paddle 형태로 변환
)

# 2) 토크나이저/이미지 프로세서 (베이스와 동일 소스 권장)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(
    base_model, padding_side="right", model_max_length=4096
)
image_preprocessor = SiglipImageProcessor.from_pretrained(base_model)

# 3) LoRA 어댑터 적용 (출력 디렉토리에서)
model = LoRAModel.from_pretrained(model=model, lora_path=lora_dir)
model.eval()

[2025-11-10 14:45:06,008] [    INFO] - Using download source: huggingface
[2025-11-10 14:45:06,009] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-11-10 14:45:06,011] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors
[2025-11-10 14:45:59,632] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-10 14:45:59,774] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 14:45:59,778] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 14:45:59,782] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 14:45:59,791] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 14:45:59,796] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 14:45:59,800] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 14:45:59,809] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 1

In [1]:
import paddle
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel

base_model = "PaddlePaddle/PaddleOCR-VL"  # 학습 시 사용한 베이스와 동일해야 함
lora_dir   = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test01"  # peft_model*.safetensors 위치
hf_out_dir = "/home/mango/ERNIE/PaddleOCR-VL-SFT-hf-merged"

# 1) 베이스 모델을 HF→Paddle 변환 로딩
model = PaddleOCRVLForConditionalGeneration.from_pretrained(
    base_model,
    convert_from_hf=True,      # 핵심: HF 포맷을 Paddle 형태로 변환
)

# 2) 토크나이저/이미지 프로세서 (베이스와 동일 소스 권장)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(
    base_model, padding_side="right", model_max_length=8192
)
image_preprocessor = SiglipImageProcessor.from_pretrained(base_model)

# 3) LoRA 어댑터 적용 (출력 디렉토리에서)
model = LoRAModel.from_pretrained(model=model, lora_path=lora_dir)

model.save_pretrained(hf_out_dir, safe_serialization=True)

image_preprocessor.save_pretrained(hf_out_dir)

/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
W1110 15:18:24.859114 1255067 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 12.6, Runtime API Version: 11.8
[2025-11-10 15:18:26,354] [    INFO] - Using download source: huggingface
[2025-11-10 15:18:26,355] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-11-10 15:18:26,356] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors
[2025-11-10 15:18:30,308] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-10 15:18:30,524] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 15:18:30,529] [    I

['/home/mango/ERNIE/PaddleOCR-VL-SFT-hf-merged/preprocessor_config.json']

In [1]:
import paddle
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel

# 1) 경로 설정
base_model = "PaddlePaddle/PaddleOCR-VL"  # 베이스 모델 허브 경로(또는 로컬 경로)
lora_dir   = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test03"  # peft_model*.safetensors 있는 디렉터리

# 2) 베이스 모델/토크나이저/이미지 프로세서 로드
model = PaddleOCRVLForConditionalGeneration.from_pretrained(base_model, convert_from_hf=True)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(base_model, padding_side="right", model_max_length=8192)
image_preprocessor = SiglipImageProcessor.from_pretrained(base_model)

# 3) LoRA 어댑터 적용 (출력 디렉터리에서)
model = LoRAModel.from_pretrained(model=model, lora_path=lora_dir)
#model.eval()

# (선택) LoRA 병합 후 고정이 필요하면, 다음 API가 지원되면 사용:
# model.merge_and_unload()  # 지원되는 경우에만

/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
W1110 18:12:57.320622 1407321 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 12.6, Runtime API Version: 11.8
[2025-11-10 18:12:58,982] [    INFO] - Using download source: huggingface
[2025-11-10 18:12:58,983] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-11-10 18:12:58,985] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors
[2025-11-10 18:13:03,142] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-10 18:13:03,309] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 18:13:03,316] [    I

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/home/mango/ERNIE/PaddleOCR-VL-SFT-test03",
    repo_id="meister1378/vl-lora"
)

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-6911988b-224df3f3516a769c77a14a45;85cd3b56-412b-4e1a-a5f6-7fa8a8120fab)

Repository Not Found for url: https://huggingface.co/api/models/meister1378/vl-lora/preupload/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Note: Creating a commit assumes that the repo already exists on the Huggingface Hub. Please use `create_repo` if it's not the case.

In [3]:
model.save_to_hf_hub(repo_id="meister1378/vl-lora", commit_message="upload lora adapter")

[2025-11-10 16:47:42,864] [    INFO] - Configuration saved in /tmp/tmph29pu8a1/config.json
[2025-11-10 16:47:42,871] [    INFO] - Configuration saved in /tmp/tmph29pu8a1/generation_config.json
[2025-11-10 16:47:48,286] [    INFO] - Model weights saved in /tmp/tmph29pu8a1/model_state.pdparams
[2025-11-10 16:47:48,287] [    INFO] - README.md not found, adding the default README.md
[2025-11-10 16:47:48,288] [    INFO] - Pushing to the meister1378/vl-lora. This might take a while


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/meister1378/vl-lora/commit/dc2b278a96ca326e78b62b4647aa0395336064dc', commit_message='upload lora adapter', commit_description='', oid='dc2b278a96ca326e78b62b4647aa0395336064dc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/meister1378/vl-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='meister1378/vl-lora'), pr_revision=None, pr_num=None)

In [4]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/home/mango/ERNIE/PaddleOCR-VL-SFT-test01",
    repo_id="meister1378/vl-lora"
)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/meister1378/vl-lora/commit/685d5f6e171caeb04ba92dbab70d131b3a9d0a33', commit_message='Upload folder using huggingface_hub', commit_description='', oid='685d5f6e171caeb04ba92dbab70d131b3a9d0a33', pr_url=None, repo_url=RepoUrl('https://huggingface.co/meister1378/vl-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='meister1378/vl-lora'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForCausalLM, AutoProcessor

model = AutoModelForCausalLM.from_pretrained("meister1378/vl-lora", resume_download=True)

/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ValueError: The repository meister1378/vl-lora contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/meister1378/vl-lora .
 You can inspect the repository content at https://hf.co/meister1378/vl-lora.
Please pass the argument `trust_remote_code=True` to allow custom code to be run.

In [4]:
from PIL import Image
import torch
import paddle
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

CHOSEN_TASK = "ocr"  # Options: 'ocr' | 'table' | 'chart' | 'formula'
PROMPTS = {
    "ocr": "OCR:",
    "table": "Table Recognition:",
    "formula": "Formula Recognition:",
    "chart": "Chart Recognition:",
}

model_path = "meister1378/vl-lora"
image_path = "000000014_경남.jpg"
image = Image.open(image_path).convert("RGB")

model = PaddleOCRVLForConditionalGeneration.from_pretrained(model_path, convert_from_hf=True).eval()
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
messages = [
    {"role": "user",         
     "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": PROMPTS[CHOSEN_TASK]},
        ]
    }
]
inputs = processor.apply_chat_template(
    messages, 
    tokenize=True, 
    add_generation_prompt=True, 	
    return_dict=True,
    return_tensors="pt"
).to(DEVICE)

outputs_ids = model.generate(**inputs, max_new_tokens=512)
outputs = processor.batch_decode(outputs_ids, skip_special_tokens=False)[0]
print(outputs)


[2025-11-10 17:20:54,650] [    INFO] - Using download source: huggingface
[2025-11-10 17:20:54,651] [    INFO] - Loading configuration file /root/.cache/huggingface/hub/models--meister1378--vl-lora/snapshots/685d5f6e171caeb04ba92dbab70d131b3a9d0a33/config.json
[2025-11-10 17:20:54,652] [    INFO] - Using download source: huggingface
[2025-11-10 17:20:54,653] [    INFO] - Loading weights file from cache at /root/.cache/huggingface/hub/models--meister1378--vl-lora/snapshots/685d5f6e171caeb04ba92dbab70d131b3a9d0a33/model_state.pdparams
[2025-11-10 17:20:54,654] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-11-10 17:20:54,715] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 17:20:54,718] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 17:20:54,724] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 17:20:54,727] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-11-10 17:20

OSError: meister1378/vl-lora does not appear to have a file named processing_ppocrvl.py. Checkout 'https://huggingface.co/meister1378/vl-lora/tree/main' for available files.

In [4]:
model.save(safe_serialization=True)

AttributeError: 'PaddleOCRVLForConditionalGeneration' object has no attribute 'save'

In [3]:
# 필요한 라이브러리 설치 (최신 버전 권장)
# !pip install -U transformers peft accelerate safetensors torch

import torch
from transformers import AutoModelForCausalLM, AutoProcessor
from peft import PeftModel

# 1. 경로 설정
base_model_path = "PaddlePaddle/PaddleOCR-VL"
lora_adapter_path = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test01"
merged_model_save_path = "/home/mango/ERNIE/PaddleOCR-VL-SFT-hf-merged"

print("1. 베이스 모델 로딩...")
# 2. 베이스 모델 로드
# trust_remote_code=True는 허깅페이스에 등록되지 않은 커스텀 모델 코드를 실행하도록 허용합니다.
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    trust_remote_code=True,
    torch_dtype=torch.float16, # 메모리 사용량을 줄이기 위해 float16 사용 (GPU 환경)
    device_map="auto"
)

print("2. LoRA 어댑터 적용...")
# 3. LoRA 어댑터 적용
# 첫 번째 인자로 로드된 `base_model` 객체를 전달합니다.
model = PeftModel.from_pretrained(base_model, lora_adapter_path)

print("3. 어댑터와 베이스 모델 병합...")
# 4. LoRA 가중치를 베이스 모델에 완전히 병합하고, 어댑터는 메모리에서 해제
model = model.merge_and_unload()

print(f"4. 병합된 모델을 '{merged_model_save_path}'에 저장...")
# 5. 병합된 모델 저장
model.save_pretrained(merged_model_save_path, safe_serialization=True)

print(f"5. 프로세서(Processor)를 '{merged_model_save_path}'에 저장...")
# 6. 프로세서(이미지/텍스트 전처리기)도 함께 저장해야 나중에 모델을 쉽게 불러올 수 있습니다.
processor = AutoProcessor.from_pretrained(base_model_path, trust_remote_code=True)
processor.save_pretrained(merged_model_save_path)

print("\n작업 완료! 이제부터는 아래와 같이 병합된 모델을 직접 로드할 수 있습니다.")
print("--------------------------------------------------------------------")
print(f"from transformers import AutoModelForCausalLM, AutoProcessor")
print(f"model = AutoModelForCausalLM.from_pretrained('{merged_model_save_path}', trust_remote_code=True)")
print(f"processor = AutoProcessor.from_pretrained('{merged_model_save_path}', trust_remote_code=True)")
print("--------------------------------------------------------------------")

1. 베이스 모델 로딩...
2. LoRA 어댑터 적용...


/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['do_qat', 'dtype', 'enable_lora_list', 'head_dim', 'lora_plus_scale', 'lora_use_mixer', 'loraga', 'lorapro', 'merge_weights', 'pissa', 'rslora', 'scaling', 'tensor_parallel_degree', 'trainable_bias', 'trainable_modules', 'use_mora', 'use_quick_lora'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


ValueError: Target modules {'.*o_proj.*', '.*spatial_linear.2.*', '.*temporal_linear.2.*', '.*down_proj.*', '.*temporal_linear.0.*', '.*spatial_linear.0.*', '.*up_gate_proj.*', '.*qkv_proj.*'} not found in the base model. Please check the target modules and try again.

In [6]:
# pip install -U transformers peft accelerate safetensors
from transformers import AutoModelForCausalLM, AutoProcessor
from peft import get_peft_model
from peft import PeftModel

base_model = "PaddlePaddle/PaddleOCR-VL"  # 학습 때 쓴 베이스와 동일
lora_dir   = "/home/mango/ERNIE/PaddleOCR-VL-SFT-test01"
hf_out_dir = "/home/mango/ERNIE/PaddleOCR-VL-SFT-hf-merged"

pre_model = AutoModelForCausalLM.from_pretrained(base_model, trust_remote_code=True)
model = PeftModel.from_pretrained(base_model, lora_dir)   # 어댑터 로드
#model = model.merge_and_unload()                     # LoRA 병합

# processor = AutoProcessor.from_pretrained(base_model, trust_remote_code=True)
# model.save_pretrained(hf_out_dir, safe_serialization=True)
# processor.save_pretrained(hf_out_dir)

ValueError: Can't find 'adapter_config.json' at '/home/mango/ERNIE/PaddleOCR-VL-SFT-test01'

In [11]:
from huggingface_hub import HfApi
api = HfApi()
api.create_repo(repo_id="meister1378/vl-lora", exist_ok=True)
# api.upload_folder(
#     repo_id="meister1378/vl-lora",
#     folder_path="/home/mango/ERNIE/PaddleOCR-VL-SFT-test01/",  # peft_model*.safetensors 등이 있는 폴더
#     commit_message="upload lora adapter",
# )

RepoUrl('https://huggingface.co/meister1378/vl-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='meister1378/vl-lora')

In [6]:
from PIL import Image
import torch
from transformers import AutoModelForCausalLM, AutoProcessor

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

CHOSEN_TASK = "ocr"  # Options: 'ocr' | 'table' | 'chart' | 'formula'
PROMPTS = {
    "ocr": "OCR:",
    "table": "Table Recognition:",
    "formula": "Formula Recognition:",
    "chart": "Chart Recognition:",
}

model_path = "meister1378/ocr-vl-lora"
image_path = "test.png"
image = Image.open(image_path).convert("RGB")

model = AutoModelForCausalLM.from_pretrained(
    model_path, trust_remote_code=True, torch_dtype=torch.bfloat16
).to(DEVICE).eval()
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

messages = [
    {"role": "user",         
     "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": PROMPTS[CHOSEN_TASK]},
        ]
    }
]
inputs = processor.apply_chat_template(
    messages, 
    tokenize=True, 
    add_generation_prompt=True, 	
    return_dict=True,
    return_tensors="pt"
).to(DEVICE)

outputs = model.generate(**inputs, max_new_tokens=1024)
outputs = processor.batch_decode(outputs, skip_special_tokens=True)[0]
print(outputs)


FileNotFoundError: [Errno 2] No such file or directory: 'test.png'

In [ ]:
meister1378/ocr-vl-lora